In [1]:
from scipy.optimize import linprog

## Q4
A paint manufacturer, PCBCo, must determine how much paint should be produced in the next four quarters. The firm estimates the demand for the next four months to be 40, 60, 75, and 25 in thousands of lbs. The firm must fully satisfy the demand in each month, i.e., no backlogging, In the beginning of period 1, PCBCo has an inventory of 10,000 lbs. The firm 

1.	has a production capacity of 40,000 lbs. each quarter with a per unit cost of $400 per thousand lbs using regular labor, 
2.	may have workers work overtime where the per unit cost increases to $450 per thousand lbs.,
3.	incurs a holding cost of $20 per thousand lbs. at the end of a period. 

How many lbs. of paint should PCBCo produce to satisfy demand while minimizing total production and inventory costs? (Hint: The inventory cost is based on the amount stored at the end of the period!)


In [2]:
# preprocessing data
'''
 x1 = q1 regular time production
 x2 = q2 regular time production
 x3 = q3 regular time production
 x4 = q4 regular time production
 x5 = q1 overtime production
 x6 = q2 overtime production
 x7 = q3 overtime production
 x8 = q4 overtime production 
 x9 = 1 (dummy)

 reg prod cost = 400
 overtime cost = 450
 carrying cost = 20

 q1Cost = x1*400+x5*450+20*(10+x1+x5-40) = 420*x1+470*x5-600 
 q2Cost = x2*400+x6*450+20*((x1+x5+10-40)+x2+x6-60) = 420*x2+470*x6+20*x1+20*x5-1800 
 q3Cost = x3*400+x7*450+20*(((x1+x5-30)+x2+x6-60)+x3+x7-75) = 420*x3+470*x7+20*x1+20*x5+20*x2+20*x6-3300 
 q4Cost = x4*400+x8*450+20*((((x1+x5-30)+x2+x6-60)+x3+x7-75)+x4+x8-25) = 420*x4+470*x8+20*x1+20*x5+20*x2+20*x6+20*x3+20*x7-3800 
 objective = q1Cost+q2Cost+q3Cost+q4Cost = 480*x1+460*x2+440*x3+420*x4+530*x5+510*x6+490*x7+470*x8-9500*x9
 
 Constrains 
 Q1 demand -> 10+x1+x5 >= 40 -> -x1-x5 <= -30
 Q2 demand -> 10+x1+x5-40+x2+x6 >= 60 -> -x1-x5-x2-x6 <= -90
 Q3 demand -> x1+x2+x5+x6-90+x3+x7 >= 75 -> -x1-x2-x3-x5-x6-x7 <= -165
 Q4 demand -> x1+x2+x3+x5+x6+x7-165+x4+x8 >= 25 -> -x1-x2-x3-x4-x5-x6-x7-x8 <= -190
'''

# objective function coefficients
# 480*x1+460*x2+440*x3+420*x4+530*x5+510*x6+490*x7+470*x8-9500*x9
obj = [480,460,440,420,530,510,490,470,-9500] 

# left hand side coefficients
lhs_ineq = [[1,0,0,0,0,0,0,0,0], 
            [0,1,0,0,0,0,0,0,0],
            [0,0,1,0,0,0,0,0,0],
            [0,0,0,1,0,0,0,0,0],
            [-1,0,0,0,-1,0,0,0,0],
            [-1,-1,0,0,-1,-1,0,0,0],
            [-1,-1,-1,0,-1,-1,-1,0,0],
            [-1,-1,-1,-1,-1,-1,-1,-1,0]
            ]  

# right hand side values
rhs_ineq = [40,     # q1 max reg hour production 
            40,     # q2 max reg hour production 
            40,     # q3 max reg hour production 
            40,     # q4 max reg hour production 
            -30,    # q1 adjusted demand 
            -90,    # q2 adjusted demand 
            -165,   # q3 adjusted demand 
            -190    # q4 adjusted demand 
            ]  
# lhs_ineq <= rhs_ineq 

# equal constrains
lhs_eq = [[0,0,0,0,0,0,0,0,1]] 
rhs_eq = [1] # x9 = 1

bnd = [(0, float("inf")),  
       (0, float("inf")), 
       (0, float("inf")), 
       (0, float("inf")), 
       (0, float("inf")), 
       (0, float("inf")), 
       (0, float("inf")), 
       (0, float("inf")), 
       (0, float("inf"))
       ]  

opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
                A_eq=lhs_eq, b_eq=rhs_eq,
                bounds=bnd,
                method="revised simplex")
opt

     con: array([0.])
     fun: 78450.0
 message: 'Optimization terminated successfully.'
     nit: 7
   slack: array([ 0.,  0.,  0., 15., 10.,  0.,  0.,  0.])
  status: 0
 success: True
       x: array([40., 40., 40., 25.,  0., 10., 35.,  0.,  1.])

## Q8

Unwind Wine company produces two types of blended wines, Ocean View and Lake Life by mixing two wines. Ocean View must contain at least 75% of type 1 wine and Lake Life must contain at least 45% of type 1 wine. Ocean View sells for \$1 per ounce and Lake Life sells for \$7 per ounce. The firm uses two types of mixing processes. The first process uses 8 ounces of raw material in one hour and 3 hours of labor and it yields 4 ounces of each type of wine. The second process uses 6 ounces of raw material, 2 hours of labor, and yields 5 ounces of type 1 and 2 ounces of type 2 wine. Unwind Wine company has 6000 ounces of raw material and 2000 hours of labor available. Write the linear mathematical model that maximizes Unwind Wine Company’s revenue. 

In [3]:
# pre processing data 
'''
 x1 = process 1
 x2 = process 2
 x3 = OV wine 1
 x4 = OV wine 2
 x5 = LL wine 1
 x6 = LL wine 2

 objective = 0*x1+0*x2+(x3+x4)*1+(x5+x6)*7 -> 0*x1+0*x2+x3+x4+7*x5+7*x6

 wine 1 yield = 4*x1+5*x2
 wine 2 yield = 4*x1+2*x2
 
 constrains 
 Wine 1 limit = x3+x5 <= 4*x1+5*x2 -> -4*x1-5*x2+x3+x5 <= 0
 Wine 2 limit = x4+x6 <= 4*x1+2*x2 -> -4*x1-2*x2+x4+x6 <= 0
 OV blend = x3 >= (x3+x4)*0.75 -> -0.25*x3+0.75*x4 <= 0
 LL blend = x5 >= (x5+x6)*0.45 -> -0.55*x5+0.45*x6 <= 0
'''

# objective function coefficients
# 0*x1+0*x2+x3+x4+7*x5+7*x6
obj = [0, 0, -1, -1, -7, -7] # maximize

# left hand side coefficients
lhs_ineq = [[8,6,0,0,0,0], 
            [3,2,0,0,0,0],
            [-4,-5,1,0,1,0], 
            [-4,-2,0,1,0,1], 
            [0,0,-0.25,0.75,0,0], 
            [0,0,0,0,-0.55,0.45], 
            ]  

# right hand side values
rhs_ineq = [6000,   # material
            2000,   # labor
            0,      # wine 1 limit 
            0,      # wine 2 limit 
            0,      # OV blend
            0,      # LL blend
            ]  
# lhs_ineq <= rhs_ineq 

# equal constrains
# lhs_eq = [[0,0,0,0,0,0,0,0,1]] 
# rhs_eq = [1] # x9 = 1

bnd = [(0, float("inf")),  
       (0, float("inf")), 
       (0, float("inf")), 
       (0, float("inf")), 
       (0, float("inf")), 
       (0, float("inf"))
       ]  

opt = linprog(c=obj, A_ub=lhs_ineq, b_ub=rhs_ineq,
                # A_eq=lhs_eq, b_eq=rhs_eq,
                bounds=bnd,
                method="revised simplex")
opt

     con: array([], dtype=float64)
     fun: -49000.0
 message: 'Optimization terminated successfully.'
     nit: 9
   slack: array([   0.,    0.,    0.,    0.,    0., 1850.])
  status: 0
 success: True
       x: array([   0., 1000.,    0.,    0., 5000., 2000.])